In [42]:
from wba_tools import *

def generate_bratteli_poly_full_db(p,q,z,K):
    db = {}   
    irreps = {}
    for k in range(p+q):
        if k<=p:
            irreps[k] = walled_brauer_irreducibles(k,0)
        else:
            irreps[k] = walled_brauer_irreducibles(p,k-p)
        
    for k in range(p+q):
        level_p = True if k < p  else False
        for Lambda in irreps[k]:
            for Mu in bratteli_descendants(Lambda,level_p):
                J = WalledBrauerAlgebra(p,q,z,K).jucys_murphy(k+1)   
                db[(Lambda,Mu)] = interpolating_polynomial_lambda_mu(J,Lambda,Mu,p,q,z,K,level_p)
    return db

def generate_bratteli_poly_db(p,q,d):
    if d == var('z'):
        K.<z> = FractionField(PolynomialRing(ZZ, 'z'))
        return generate_bratteli_poly_full_db(p,q,z,K)
    
    db = {}  
    irreps = {}
    for k in range(p+q):
        if k<=p:
            irreps[k] = walled_brauer_irreducibles_d(k,0,d)
        else:
            irreps[k] = walled_brauer_irreducibles_d(p,k-p,d)
        
    for k in range(p+q):
        level_p = True if k < p else False
        for Lambda in irreps[k]:
            for Mu in bratteli_descendants_d(Lambda,level_p,d):
                J = WalledBrauerAlgebra(p,q,d,QQ).jucys_murphy(k+1)
                #Interpolating polynomial of the family A
                #db[(Lambda,Mu)] = interpolating_polynomial_lambda_mu_d(J,Lambda,Mu,p,q,d,QQ,level_p)
                #Interpolating polynomial of the family B
                db[(Lambda,Mu)] = interpolating_polynomial_lambda_mu(J,Lambda,Mu,p,q,d,QQ,level_p)
    return db

def generate_primitive_db(db_bratteli_poly,p,q,d):
    db = {}  
    if d == var('z'):
        for Lambda in walled_brauer_irreducibles(p,q): 
            Paths = gelfand_tsetlin_basis(Lambda,p,q)
            for path in Paths:
                eps = db_bratteli_poly[(path[0],path[1])]
                for i in range(1,len(path)-1):
                    eps *= db_bratteli_poly[(path[i],path[i+1])]
                db[tuple(path)] = eps
    else:
        for Lambda in walled_brauer_irreducibles_d(p,q,d): 
            Paths = gelfand_tsetlin_basis_d(Lambda,p,q,d)
            for path in Paths:
                eps = db_bratteli_poly[(path[0],path[1])]
                for i in range(1,len(path)-1):
                    eps *= db_bratteli_poly[(path[i],path[i+1])]
                db[tuple(path)] = eps

    return db

def generate_central_db(db_bratteli_poly,p,q,d):
    db = {}
    
    Lambda = walled_brauer_irreducibles(0,0)[0]
    db[0] = {}
    if d == var('z'):
        K.<z> = FractionField(PolynomialRing(ZZ, 'z'))
        db[0][Lambda] = WalledBrauerAlgebra(p,q,z,K).one()
    else: 
        db[0][Lambda] = WalledBrauerAlgebra(p,q,d,QQ).one()
            
    for k in range(1,p+q+1):  
        if k <= p:
            pk, qk = k, 0
        else: 
            pk, qk = p, k-p
        db[k] = {}
        
        if d == var('z'):
            for Mu in walled_brauer_irreducibles(pk,qk):
                db[k][Mu] = sum([db_bratteli_poly[(Lambda,Mu)]*db[k-1][Lambda] for Lambda in bratteli_antecedents(Mu,pk,qk)])
        else:
            for Mu in walled_brauer_irreducibles_d(pk,qk,d):
                db[k][Mu] = sum([db_bratteli_poly[(Lambda,Mu)]*db[k-1][Lambda] for Lambda in bratteli_antecedents_d(Mu,pk,qk,d)])

    return db

In [71]:
#test
d = var('z')
d = 2
p = 2
q = 2

%time db_bratteli_poly = generate_bratteli_poly_db(p,q,d)
%time db_central = generate_central_db(db_bratteli_poly,p,q,d)
%time db_primitive = generate_primitive_db(db_bratteli_poly,p,q,d)

CPU times: user 19.3 ms, sys: 14.7 ms, total: 33.9 ms
Wall time: 72.5 ms
CPU times: user 9.34 ms, sys: 500 µs, total: 9.84 ms
Wall time: 10.6 ms
CPU times: user 12.5 ms, sys: 788 µs, total: 13.3 ms
Wall time: 14 ms


In [32]:
# print([psi_wb(x,d,p,q,QQ)**2==psi_wb(x,d,p,q,QQ) for x in db_primitive.values()])
# print([x**2==x for x in db_primitive.values()])
# print(sum(db_primitive.values()))
# print(psi_wb(sum(db_primitive.values()),p,q,d,QQ))

### Generate primitive idmepotents for $ 1 \leq p+q \leq n$

In [34]:
n = 7
for k in range(1,n+1):
    for p in range(0,k+1):
        q = k - p
        d = var('z')
        print(p,q)
        %time db_bratteli_poly = generate_bratteli_poly_db(p,q,d)
        %time db_primitive = generate_primitive_db(db_bratteli_poly,p,q,d)
        print()
        file = open("./wba_idempotents_db/primitive_%d_%d.txt"%(p,q), "w")
        file.write(str(db_primitive))
        file.close()

        for d in range(2,p+q):
            db_bratteli_poly = generate_bratteli_poly_db(p,q,d)
            db_primitive = generate_primitive_db(db_bratteli_poly,p,q,d)
            file = open("./wba_idempotents_db/primitive_%d_%d_%d.txt"%(p,q,d), "w")
            file.write(str(db_primitive))
            file.close()

0 1
CPU times: user 22 ms, sys: 67.8 ms, total: 89.8 ms
Wall time: 217 ms
CPU times: user 360 µs, sys: 229 µs, total: 589 µs
Wall time: 1.12 ms

1 0
CPU times: user 962 µs, sys: 399 µs, total: 1.36 ms
Wall time: 2.3 ms
CPU times: user 146 µs, sys: 0 ns, total: 146 µs
Wall time: 150 µs

0 2
CPU times: user 6.25 ms, sys: 2.8 ms, total: 9.05 ms
Wall time: 16.2 ms
CPU times: user 617 µs, sys: 0 ns, total: 617 µs
Wall time: 621 µs

1 1
CPU times: user 5.18 ms, sys: 645 µs, total: 5.83 ms
Wall time: 7.67 ms
CPU times: user 584 µs, sys: 6 µs, total: 590 µs
Wall time: 600 µs

2 0
CPU times: user 5.69 ms, sys: 729 µs, total: 6.42 ms
Wall time: 6.89 ms
CPU times: user 577 µs, sys: 0 ns, total: 577 µs
Wall time: 581 µs

0 3
CPU times: user 13.1 ms, sys: 1.09 ms, total: 14.2 ms
Wall time: 17 ms
CPU times: user 2.62 ms, sys: 376 µs, total: 3 ms
Wall time: 3.88 ms

1 2
CPU times: user 12.6 ms, sys: 191 µs, total: 12.8 ms
Wall time: 12.8 ms
CPU times: user 2.79 ms, sys: 85 µs, total: 2.87 ms
Wall tim

KeyboardInterrupt: 

In [1]:
db_primitive

NameError: name 'db_primitive' is not defined